In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.appName("preprocessing of taxi data")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/01 08:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [24]:
merchant_sdf = spark.read.parquet("../data/tables/tbl_merchants.parquet")
consumer_detail_sdf = spark.read.parquet("../data/tables/consumer_user_details.parquet")
consumer_sdf = spark.read.option("header",True) \
                         .option("inferSchema",True) \
                         .options(delimiter='|') \
                         .csv("../data/tables/tbl_consumer.csv")
transaction_sdf = spark.read.parquet("../data/tables/transactions_20210228_20210827_snapshot")

# Transaction dataset
- should be our main focus in terms on analysing the number of transactions related to each customer and each merchant
- could be joined to consumer_detail by using user_id and consumer_id with the help of tbl_consumer
- join merchant on merchant abn (should be unique by personal knowledge)

In [8]:
transaction_sdf.orderBy(F.column("order_datetime"))


user_id,merchant_abn,dollar_value,order_id,order_datetime
11,20692490685,196.93916081228323,09bc8dd6-419f-4cb...,2021-02-28
5,83177825742,66.66426160206629,43e3b3fe-791b-47f...,2021-02-28
18495,66667026714,48.3228941934147,ca150cbf-6e34-489...,2021-02-28
18489,43186523025,98.14878546968934,9008a98e-1b02-4de...,2021-02-28
18491,64974914166,130.12601873970038,4bc15338-83eb-43d...,2021-02-28
18493,67979471799,53.03432496202065,dd61feff-5aa3-43d...,2021-02-28
11,98269572896,129.46280909485031,8f5d0cab-8055-435...,2021-02-28
18485,62191208634,79.13140006851712,9e18b913-0465-4fd...,2021-02-28
2,80779820715,48.12397733548124,cd09bdd6-f56d-489...,2021-02-28
18490,93558142492,232.83335268750145,2bda0665-796f-4f2...,2021-02-28


In [30]:
merchant_sdf

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


In [29]:
merchant_sdf.select(F.col("tags")).take(1)

[Row(tags='((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))')]

In [11]:
consumer_detail_sdf

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [25]:
consumer_sdf

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


In [47]:
consumer_detail_sdf.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- consumer_id: long (nullable = true)



In [45]:
transaction_sdf.select(F.col("user_id")).count()

3643266

In [34]:
consumer_sdf.filter(consumer_sdf["consumer_id"].isNull()).count()

0

In [35]:
consumer_detail_sdf.filter(consumer_detail_sdf["consumer_id"].isNull()).count()

0

In [36]:
consumer_detail_sdf.filter(consumer_detail_sdf["user_id"].isNull()).count()

0